In [1]:
# # install simpletransformers
# !pip install -U simpletransformers


# # check installed version
# !pip freeze | grep simpletransformers
# # simpletransformers==0.28.2

## Step 1:- Loading datasets and importing libraries

In [ ]:
import pandas as pd
train_df = pd.read_csv('/content/train.csv',sep=";", encoding="utf8")
train_df.head()

,Unnamed: 0,Id,Impact,Urgency,IncidentType,ServiceProcessed,MailSubject,MailTextBody,ManualGroups
0,3554,INC000010587669,4-Minor/Localized,2-High,Failure,EDA_S_BA_2FA,smart card blockiert MD5:2225e4a30a5b7e592ba34...,lieber helpdesk sie haben mir den computer fr...,NaN
1,3553,INC000010585556,4-Minor/Localized,4-Low,Service Request,EDA_S_Order Management,FW: Webcam MD5:f757a42a9bdb1dfd9c8427e588586ca8,liebe kolleginnen und kollegen ich möchte für...,NaN
2,3552,INC000010585519,4-Minor/Localized,4-Low,Failure,EDA_S_Peripheriegeräte,FW: IT Support heute Nachmittag MD5:bc594d6bfd...,liebe kolleginnen und kollegen es handelt sic...,NaN
3,3549,INC000010582950,4-Minor/Localized,4-Low,Service Request,EDA_S_Benutzerunterstützung,Botschaft Warschau - Administratoren-Zugang f...,"sehr geehrte damen und herren, die botschaft ...",Benutzeranleitungen_Telefonie
4,3548,INC000010582919,4-Minor/Localized,4-Low,Service Request,EDA_S_Order Management,FW: Commande d'un écran MD5:c2b8cb3281fed4e2d4...,liebe kolleginnen und kollegen ich möchte für...,NaN


In [ ]:
test_df = pd.read_csv('/content/test_reduced.csv',sep=";", encoding="utf8")
test_df.head()

,Unnamed: 0,Id,MailSubject,MailTextBody
0,2122,INC000009681999,Traveler 0 1 2 3 austauschen (altes Modell),guten tag wir haben im 5. stock an der bg32 e...
1,202,INC000009004056,Orchestrator läuft nicht,hallo helpdesk orchestrator läuft nicht. d...
2,536,INC000009244944,Ausschalten des Bildschirms eines Laptops,liebe kolleginnen und kollegen mein kolleg...
3,22,INC000008891561,Skype,bitte den neuen mitarbeiter daniel kleis kln i...
4,2249,INC000009702828,Dell Laptop Batterie,liebes helpdesk die batterie meines dell lap...


In [ ]:
train_df.loc[train_df['MailSubject'].isnull(),'MailSubject']=''

In [ ]:
train_df.ServiceProcessed.nunique()

55

In [ ]:
train_df.ServiceProcessed.value_counts()/train_df.shape[0]

EDA_S_Order Management                  0.078735
EDA_S_BA_Mailbox                        0.075923
EDA_S_APS_OS_BasisSW                    0.073111
EDA_S_Mobile Kommunikation              0.070650
EDA_ANW_SysP eDoc                       0.060808
EDA_S_APS_PC                            0.052373
EDA_S_BA_UCC_Benutzertelefonie          0.051670
EDA_S_Netzdrucker                       0.040070
EDA_S_BA_2FA                            0.035149
EDA_S_APS_Monitor                       0.034798
EDA_ANW_Intranet/Collaboration EDA      0.031634
EDA_S_Zusätzliche Software              0.029174
EDA_S_BA_Account                        0.028120
EDA_S_Netzwerk Ausland                  0.026714
EDA_S_APS_Peripherie                    0.026714
EDA_ANW_SAP Services                    0.026362
EDA_S_Betrieb Übermitttlungssysteme     0.026362
EDA_S_Benutzerunterstützung             0.025659
EDA_S_BA_UCC_IVR                        0.024253
EDA_S_Peripheriegeräte                  0.022496
EDA_ANW_CH@World (MO

In [ ]:
print(train_df.shape)
print(test_df.shape)
print(train_df.Id.nunique())
print(test_df.Id.nunique())

(2845, 9)
(712, 4)
2845
712


In [ ]:
train_df['Text']=train_df['MailSubject']+" "+train_df['MailTextBody']
test_df['Text']=test_df['MailSubject']+" "+test_df['MailTextBody']
test_df.head(3)

,Unnamed: 0,Id,MailSubject,MailTextBody,Text
0,2122,INC000009681999,Traveler 0 1 2 3 austauschen (altes Modell),guten tag wir haben im 5. stock an der bg32 e...,Traveler 0 1 2 3 austauschen (altes Modell) gu...
1,202,INC000009004056,Orchestrator läuft nicht,hallo helpdesk orchestrator läuft nicht. d...,Orchestrator läuft nicht hallo helpdesk or...
2,536,INC000009244944,Ausschalten des Bildschirms eines Laptops,liebe kolleginnen und kollegen mein kolleg...,Ausschalten des Bildschirms eines Laptops lieb...


## Step 2:- Basic preprocessing done on the text part for train and test

In [ ]:
train_df['Text']=train_df['Text'].str.replace('\\r',' ')
train_df['Text']=train_df['Text'].str.replace('\\n','')
train_df['Text']=train_df['Text'].str.replace('\r','')
train_df['Text']=train_df['Text'].str.replace('\n','')
train_df['Text']=train_df['Text'].str.replace('  ','')
train_df['Text']=train_df['Text'].str.lower()
train_df['Text']=train_df['Text'].str.replace('&nbsp;','')
train_df['Text']=train_df['Text'].str.strip()

In [ ]:
test_df['Text']=test_df['Text'].str.replace('\\r',' ')
test_df['Text']=test_df['Text'].str.replace('\\n','')
test_df['Text']=test_df['Text'].str.replace('\r','')
test_df['Text']=test_df['Text'].str.replace('\n','')
test_df['Text']=test_df['Text'].str.replace('  ','')
test_df['Text']=test_df['Text'].str.lower()
test_df['Text']=test_df['Text'].str.replace('&nbsp;','')
test_df['Text']=test_df['Text'].str.strip()

In [ ]:
train_df['Text_length']=train_df['Text'].str.len()
test_df['Text_length']=test_df['Text'].str.len()

In [ ]:
print(train_df['Text_length'].describe())
print(test_df['Text_length'].describe())

count     2845.000000
mean       623.125483
std       1426.332357
min         42.000000
25%        269.000000
50%        392.000000
75%        611.000000
max      43188.000000
Name: Text_length, dtype: float64
count     712.000000
mean      521.792135
std       481.612057
min        68.000000
25%       263.000000
50%       386.500000
75%       572.500000
max      5658.000000
Name: Text_length, dtype: float64


In [ ]:
class_list=train_df.ServiceProcessed.unique().tolist()
train_df['pred_class']=train_df.apply(lambda x:  class_list.index(x['ServiceProcessed']),axis=1)

In [ ]:
train_df1=train_df[['Text','pred_class']]
train_df1['Text']=train_df1['Text'].astype(str)
train_df1.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Text,pred_class
0,smart card blockiert md5:2225e4a30a5b7e592ba34...,0
1,fw: webcam md5:f757a42a9bdb1dfd9c8427e588586ca...,1
2,fw: it support heute nachmittag md5:bc594d6bfd...,2
3,botschaft warschau -administratoren-zugang für...,3
4,fw: commande d'un écran md5:c2b8cb3281fed4e2d4...,1


### Step 3:- Setting up a validation dataset 

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df1 = train_test_split(train_df1, test_size=0.05)

print('train shape: ',train_df.shape)
print('test shape: ',test_df1.shape)

train shape:  (2702, 2)
test shape:  (143, 2)


### Step 4:- Calling the model with parameter tuned based on Analysis done above

In [ ]:
from simpletransformers.classification import ClassificationModel

model = ClassificationModel("xlmroberta", "xlm-roberta-large-finetuned-conll03-german", use_cuda=True,num_labels=55, args={'train_batch_size':2,
                                                                         'reprocess_input_data': True,
                                                                         'overwrite_output_dir': True,
                                                                         'fp16': False,
                                                                         'do_lower_case': False,
                                                                         'num_train_epochs': 3,
                                                                         'max_seq_length': 512,
                                                                         'regression': False,
                                                                         'manual_seed': 42,
                                                                         "learning_rate":1e-5,
                                                                         'weight_decay':0,
                                                                         "save_eval_checkpoints": False,
                                                                         "save_model_every_epoch": False,
                                                                         "silent": False})


Some weights of the model checkpoint at xlm-roberta-large-finetuned-conll03-german were not used when initializing XLMRobertaForSequenceClassification: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large-finetuned-conll03-german and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:276: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  f"use_multiprocessing automatically disabled as {model_type}"


##### Step 5:- Training the model 

In [ ]:
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:377: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(4053, 1.8073764436322413)

## Step 6:- Validating the model on the validation set 

In [ ]:
from sklearn.metrics import f1_score, accuracy_score


test_df1['Text']=test_df1['Text'].astype(str)
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)

result

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:896: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


{'acc': 0.6713286713286714,
 'eval_loss': 1.511341439353095,
 'f1': 0.6713286713286714,
 'mcc': 0.6551725567108374}

In [ ]:
test_df.head(2)

,Unnamed: 0,Id,MailSubject,MailTextBody,Text,Text_length
0,2122,INC000009681999,Traveler 0 1 2 3 austauschen (altes Modell),guten tag wir haben im 5. stock an der bg32 e...,traveler 0 1 2 3 austauschen (altes modell) gu...,538
1,202,INC000009004056,Orchestrator läuft nicht,hallo helpdesk orchestrator läuft nicht. d...,orchestrator läuft nicht hallo helpdesk orches...,232


### Final step :- Predicting the outcome on the test dataset given and preparing the csv for submission on kaggle

In [ ]:
preds=model.predict(test_df['Text'])[0]
len(preds)

712

In [ ]:
output_df=pd.DataFrame(test_df['Id'])
output_df['Predicted']=preds
output_df.head()

,Id,Predicted
0,INC000009681999,14
1,INC000009004056,18
2,INC000009244944,14
3,INC000008891561,13
4,INC000009702828,14


In [ ]:
######### Analysing the distribution of our outputs with the given training set classes
output_df['Predicted']=output_df['Predicted'].apply(lambda x:  class_list[x])
output_df['Predicted'].value_counts()/output_df.shape[0]

EDA_S_Order Management                 0.103933
EDA_S_BA_Mailbox                       0.095506
EDA_S_APS_OS_BasisSW                   0.073034
EDA_S_APS_PC                           0.071629
EDA_ANW_SysP eDoc                      0.068820
EDA_S_BA_UCC_Benutzertelefonie         0.058989
EDA_S_Mobile Kommunikation             0.057584
EDA_S_Netzdrucker                      0.051966
EDA_S_APS_Monitor                      0.044944
EDA_ANW_Intranet/Collaboration EDA     0.039326
EDA_ANW_SAP Services                   0.036517
EDA_S_BA_2FA                           0.036517
EDA_S_BA_Account                       0.035112
EDA_S_Zusätzliche Software             0.030899
EDA_S_Betrieb Übermitttlungssysteme    0.029494
EDA_S_APS_Peripherie                   0.028090
EDA_S_BA_UCC_IVR                       0.022472
EDA_S_Netzwerk Ausland                 0.021067
EDA_ANW_CH@World (MOSS)                0.016854
EDA_S_BA_Datenablage                   0.016854
EDA_S_Benutzerunterstützung            0

In [ ]:
output_df.to_csv("xlmroberta_dropna.csv",index=False)

############ This is what we submitted